## Implementation of the Gale-Shapley Algorithm in Python

_Generates random preferences for men and women and confirms that the Gale-Shapley algorithm always returns a stable matching, providing a
clear explanation and validation of stability in the generated matchings._

In [2]:
women = ['Amanda', 'Barbara', 'Charlotte', 'Diana']
men = ['Adam', 'Bob', 'Charlie', 'David']

pref_dict_w = {
    'Amanda': ['Adam', 'Bob', 'Charlie', 'David'],
    'Barbara': ['Bob', 'David', 'Adam', 'Charlie'],
    'Charlotte': ['Charlie', 'Bob', 'Adam', 'David'],
    'Diana': ['David', 'Adam', 'Bob', 'Charlie'],
}

pref_dict_m = {
    'Adam': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    'Bob': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    'Charlie': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    'David': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
}


In [11]:
def pref_to_rank(pref_list):
    return {
        a: {b: i for i, b in enumerate(a_pref)} for a, a_pref in pref_list.items()
    }

pref_rank_w = pref_to_rank(pref_dict_w)
pref_rank_m = pref_to_rank(pref_dict_m)

In [12]:
pref_dict_w, pref_dict_m


({'Amanda': ['Adam', 'Bob', 'Charlie', 'David'],
  'Barbara': ['Bob', 'David', 'Adam', 'Charlie'],
  'Charlotte': ['Charlie', 'Bob', 'Adam', 'David'],
  'Diana': ['David', 'Adam', 'Bob', 'Charlie']},
 {'Adam': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
  'Bob': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
  'Charlie': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
  'David': ['Amanda', 'Barbara', 'Charlotte', 'Diana']})

In [13]:
pref_rank_w, pref_rank_m

({'Amanda': {'Adam': 0, 'Bob': 1, 'Charlie': 2, 'David': 3},
  'Barbara': {'Bob': 0, 'David': 1, 'Adam': 2, 'Charlie': 3},
  'Charlotte': {'Charlie': 0, 'Bob': 1, 'Adam': 2, 'David': 3},
  'Diana': {'David': 0, 'Adam': 1, 'Bob': 2, 'Charlie': 3}},
 {'Adam': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3},
  'Bob': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3},
  'Charlie': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3},
  'David': {'Amanda': 0, 'Barbara': 1, 'Charlotte': 2, 'Diana': 3}})

In [17]:
from collections import namedtuple

pair = namedtuple('pair', ['woman', 'man'])
pair('Amanda', 'Adam')

pair(woman='Amanda', man='Adam')

### Brute-Forcing It

In [23]:
from itertools import permutations

def stable_match_forced(
        *, women, men, pref_rank_w, pref_rank_m
):
    '''
    Solve stable matching problem using brute force

    women: set[str] set of women
    men: set[str] set of men
    pref_rank_w: dict[str, dict[str, int]] preference rank
    pref_rank_m: dict[str, dict[str, int]] preference rank
    '''

    w_rank = pref_to_rank(pref_rank_w)
    m_rank = pref_to_rank(pref_rank_m)

    w_seq = tuple(women)
    matching = (
        [
            pair(w, m)
            for w, m in zip(w_seq, m_seq)
        ]
        for m_seq in permutations(men)
    )

    for match in matching:
        match_w = {pair.woman: pair for pair in match}
        match_m = {pair.man: pair for pair in match}

        unstable = any(
            (
                w_rank[w][m] < w_rank[w][match_w[w].man] and
                m_rank[m][w] < m_rank[m][match_m[m].woman]
            )
            for w in women
            for m in men
            if w != match_m[m].woman
            if m != match_w[w].man
        )
        if not unstable:
            return match

In [24]:
stable_match_forced(
    women = {'Amanda', 'Barbara', 'Charlotte', 'Diana'},
    men = {'Adam', 'Bob', 'Charlie', 'David'},
    pref_rank_w = {
        'Amanda': ['Adam', 'Bob', 'Charlie', 'David'],
        'Barbara': ['Bob', 'David', 'Adam', 'Charlie'],
        'Charlotte': ['Charlie', 'Bob', 'Adam', 'David'],
        'Diana': ['David', 'Adam', 'Bob', 'Charlie'],
    },
    pref_rank_m = {
        'Adam': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'Bob': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'Charlie': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
        'David': ['Amanda', 'Barbara', 'Charlotte', 'Diana'],
    }
)

[pair(woman='Diana', man='David'),
 pair(woman='Barbara', man='Bob'),
 pair(woman='Charlotte', man='Charlie'),
 pair(woman='Amanda', man='Adam')]

In [25]:
%timeit stable_match_forced(women={'Amanda', 'Barbara', 'Charlotte', 'Diana'}, men={'Adam', 'Bob', 'Charlie', 'David'}, pref_rank_w=pref_rank_w, pref_rank_m=pref_rank_m)

24.6 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
